In [1]:
import numpy as np
import pandas as pd
import import_ipynb
from utils import show

importing Jupyter notebook from utils.ipynb


In [2]:
def lst_final(data):
    shifted_data_zero = np.concatenate((data[1:],[0]))
    zero_range = np.divide(shifted_data_zero, data)
    zero_range = np.concatenate(([data.iloc[0]], zero_range))-1
    zero_range = zero_range[:-1]
    iszero = np.concatenate(([0],
                             np.greater(zero_range,0).view(np.int8),
                            [0]))
    absdiff = np.abs(np.diff(iszero))
    lst_range = np.where(absdiff==1)[0].reshape(-1,2)
    return lst_range

In [55]:
def individual_return(data):
    shifted_data_zero = np.concatenate((data[1:],[0]))
    zero_range = np.divide(shifted_data_zero, data)
    zero_range = np.concatenate(([data.iloc[0]], zero_range))-1
    zero_range = zero_range[:-1]
    return zero_range

In [3]:
df = pd.DataFrame(np.random.uniform(0.,2.5,(20,5)),columns=list('ABCDE')).cumprod(axis=0)
df.index = np.arange(20,40)
df


,A,B,C,D,E
20,2.027965,0.791905,1.443009,1.283140,1.729922
21,0.967235,0.938861,1.829655,0.449408,2.271486
22,0.619497,0.964379,3.446579,0.733403,0.117708
23,0.289735,2.315211,3.622004,1.510178,0.022990
24,0.127752,0.890473,1.366027,1.964897,0.051331
25,0.004547,0.805904,1.163724,2.687660,0.079489
26,0.004330,0.898557,1.280450,2.453138,0.077770
27,0.002195,1.361080,1.756135,6.125608,0.171135
28,0.000280,1.433208,0.669613,5.597979,0.134778
29,0.000608,2.164311,1.168731,12.604328,0.091608


In [62]:
ir = df.apply(individual_return,axis=0)

In [63]:
ir

,A,B,C,D,E
20,1.027965,-0.208095,0.443009,0.283140,0.729922
21,-0.523052,0.185573,0.267945,-0.649759,0.313057
22,-0.359518,0.027180,0.883731,0.631931,-0.948180
23,-0.532306,1.400726,0.050899,1.059139,-0.804683
24,-0.559072,-0.615382,-0.622853,0.301103,1.232723
25,-0.964404,-0.094971,-0.148096,0.367837,0.548551
26,-0.047917,0.114967,0.100304,-0.087259,-0.021617
27,-0.493119,0.514741,0.371498,1.497050,1.200521
28,-0.872243,0.052993,-0.618700,-0.086135,-0.212448
29,1.169159,0.510116,0.745381,1.251585,-0.320301


In [4]:
rst = []
#for i, data in enumerate(df.T):
for fundname in df.columns:
    #fundname = df.columns[i]
    data = df[fundname]
    single_rst = lst_final(data)
    rst.append(pd.DataFrame(single_rst, columns=[[fundname,fundname],['start','end']]))
range_df = pd.concat(rst,axis=1,join='outer')
range_df.columns.names = ['FUND','RANGE']
range_df.index.names = ['idx']
range_df

FUND      A           B           C         D         E    
RANGE start   end start   end start end start end start end
idx                                                        
0       0.0   1.0   1.0   4.0     0   4     0   1     0   2
1       9.0  13.0   6.0  10.0     6   8     2   6     4   6
2      14.0  18.0  11.0  12.0     9  12     7   8     7   8
3      19.0  20.0  13.0  15.0    13  15     9  11    10  12
4       NaN   NaN  16.0  19.0    16  17    12  16    13  17
5       NaN   NaN   NaN   NaN    18  19    17  20    18  20

In [43]:
def get_lst_return(df,range_df):
    len_range_df = range_df.diff(axis=1).stack()[::2]
    lst_mask = len_range_df == np.max(len_range_df,axis=0)
    lst_df = range_df[lst_mask]
    
    result_ls = []
    for fdname in lst_mask.columns:
        single_df = pd.DataFrame(columns=['start','end','LEN','RETURN'])
        for i in lst_df[fdname].dropna().index:
            start = lst_df[fdname].iloc[i]['start'].astype(int)
            pre_start = start - 1 if start != 0 else None
            end = lst_df[fdname].iloc[i]['end'].astype(int)
            real_end = end - 1
            #print(type(start))
            #print(df[fdname].iloc[start-1])
            if pre_start:
                value = df[fdname].iloc[real_end] / df[fdname].iloc[pre_start] -1
            else:
                value = df[fdname].iloc[real_end] - 1
            single_ser = pd.Series({'start':df.index[start],
                                    'end':df.index[real_end],
                                    'LEN':end - start,
                                    'RETURN':value})
            single_df = single_df.append(single_ser,ignore_index=True)
            single_df = single_df[single_df['RETURN']==single_df['RETURN'].max()]
        single_df.columns = [[fdname,fdname,fdname,fdname],
                             ['start','end','LEN','RETURN']]
        result_ls.append(single_df)
    result_df = pd.concat(result_ls,axis=1)
    if result_df.isnull().any().any():
        result_df = result_df.fillna(method='ffill').dropna(how='any', axis=0)
    return result_df

In [44]:
rd = get_lst_return(df, range_df)

In [45]:
rd

A                           B                          C             \
  start   end  LEN     RETURN start   end  LEN    RETURN start   end  LEN   
1  34.0  37.0  4.0  12.211803  26.0  29.0  4.0  1.685569  20.0  23.0  4.0   

                D                          E                       
     RETURN start   end  LEN    RETURN start   end  LEN    RETURN  
1  2.622004  22.0  25.0  4.0  4.980447  33.0  36.0  4.0  12.35579

In [29]:
rd.fillna(method='ffill').dropna(how='any', axis=0)

A                           B                          C             \
  start   end  LEN     RETURN start   end  LEN    RETURN start   end  LEN   
1  34.0  37.0  4.0  12.211803  26.0  29.0  4.0  1.685569  20.0  23.0  4.0   

                D                          E                       
     RETURN start   end  LEN    RETURN start   end  LEN    RETURN  
1  2.622004  22.0  25.0  4.0  4.980447  33.0  36.0  4.0  12.35579

In [50]:
rd.iloc[:,3::4]

,A,B,C,D,E
,RETURN,RETURN,RETURN,RETURN,RETURN
1,12.211803,1.685569,2.622004,4.980447,12.35579


In [ ]:
len_range_df = range_df.diff(axis=1).stack()[::2]
len_range_df

In [ ]:
lst_mask = len_range_df == np.max(len_range_df,axis=0)
#lst_mask
lst_df = range_df[lst_mask]
lst_df

In [ ]:
lst_mask

In [ ]:
result_ls = []
for fdname in lst_mask.columns:
    single_df = pd.DataFrame(columns=['start','end','LEN','RETURN'])
    for i in lst_df[fdname].dropna().index:
        #idx = i,
        
        start = lst_df[fdname].iloc[i]['start'].astype(int)
        pre_start = start - 1 if start != 0 else None
        end = lst_df[fdname].iloc[i]['end'].astype(int)
        real_end = end - 1
        #print(type(start))
        #print(df[fdname].iloc[start-1])
        if pre_start:
            value = df[fdname].iloc[real_end] / df[fdname].iloc[pre_start] -1
        else:
            value = df[fdname].iloc[real_end] - 1
        single_ser = pd.Series({'start':df.index[start],
                                'end':df.index[real_end],
                                'LEN':end - start,
                                'RETURN':value})
        single_df = single_df.append(single_ser,ignore_index=True)
        single_df = single_df[single_df['RETURN']==single_df['RETURN'].max()]
    single_df.columns = [[fdname,fdname,fdname,fdname],
                         ['start','end','LEN','RETURN']]
    result_ls.append(single_df)
result_df = pd.concat(result_ls,axis=1)
#         print('{}=>start:{}----end:{}=>value:{}'.format(fdname,start,
#                                                       real_end,value))
#         print('------------------')
        

In [ ]:
result_df